# Wrangle and Analyze Data
by Victoria Perez Mola

## Table of contents

* Introduction
* Gathering data
    * WeRateDogs Twitter archive
    * Tweet image predictions
    * Twitter count & favs data
* Assessing data
    * Quality
    * Tidiness
* Cleaning data
* Storing, Analyzing, and Visualizing
* Insights & visualization
    * Insight 1
    * Insight 2
    * Insight 3
    
## Introduction

The project consists on wrangle WeRateDogs Twitter data to create interesting and trustworthy analyses and visualizations. 
The Twitter archive is great, but it only contains very basic tweet information. Additional gathering, then assessing and cleaning is required for "Wow!"-worthy analyses and visualizations.


In [5]:
# import libraries
import requests 
import pandas as pd
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer
import numpy as np

## Gathering Data

The data for the analysis consists in three datasets: 

### 1. Enhanced Twitter Archive

The WeRateDogs Twitter archive contains basic tweet data for all 5000+ of their tweets. 
It contains each tweet's text, used to extract rating, dog name, and dog "stage" (i.e. doggo, floofer, pupper, and puppo) to make this Twitter archive "enhanced." 
Out of the 5000+ tweets, it contains tweets with ratings only (there are 2356).

In [133]:
# import the WeRateDogs Twitter archive
df_archive = pd.read_csv('Data/twitter-archive-enhanced.csv')

df_archive.head()

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0  892420643555336193                    NaN                  NaN   
1  892177421306343426                    NaN                  NaN   
2  891815181378084864                    NaN                  NaN   
3  891689557279858688                    NaN                  NaN   
4  891327558926688256                    NaN                  NaN   

                   timestamp  \
0  2017-08-01 16:23:56 +0000   
1  2017-08-01 00:17:27 +0000   
2  2017-07-31 00:18:03 +0000   
3  2017-07-30 15:58:51 +0000   
4  2017-07-29 16:00:24 +0000   

                                              source  \
0  <a href="http://twitter.com/download/iphone" r...   
1  <a href="http://twitter.com/download/iphone" r...   
2  <a href="http://twitter.com/download/iphone" r...   
3  <a href="http://twitter.com/download/iphone" r...   
4  <a href="http://twitter.com/download/iphone" r...   

                                                text  retweeted_status_id  \
0  This is Phineas. He's a mystical boy. Only eve...                  NaN   
1  This is Tilly. She's just checking pup on you....                  NaN   
2  This is Archie. He is a rare Norwegian Pouncin...                  NaN   
3  This is Darla. She commenced a snooze mid meal...                  NaN   
4  This is Franklin. He would like you to stop ca...                  NaN   

   retweeted_status_user_id retweeted_status_timestamp  \
0                       NaN                        NaN   
1                       NaN                        NaN   
2                       NaN                        NaN   
3                       NaN                        NaN   
4                       NaN                        NaN   

                                       expanded_urls  rating_numerator  \
0  https://twitter.com/dog_rates/status/892420643...                13   
1  https://twitter.com/dog_rates/status/892177421...                13   
2  https://twitter.com/dog_rates/status/891815181...                12   
3  https://twitter.com/dog_rates/status/891689557...                13   
4  https://twitter.com/dog_rates/status/891327558...                12   

   rating_denominator      name doggo floofer pupper puppo  
0                  10   Phineas  None    None   None  None  
1                  10     Tilly  None    None   None  None  
2                  10    Archie  None    None   None  None  
3                  10     Darla  None    None   None  None  
4                  10  Franklin  None    None   None  None

### 2. Additional Data via the Twitter API

Using the tweet IDs in the WeRateDogs Twitter archive, query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called tweet_json.txt file. Each tweet's JSON data should be written to its own line. Then read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count. 

In [4]:
# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
# These are hidden to comply with Twitter's API terms and conditions

consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

Since the tweets belong to the year 2017, is possible that some don't exist anymore. I'll check the status to have an idea about what to expect. 

In [5]:
#Download Tweepy status object based on Tweet ID and store in list
valid_tweet_ids = []

# save Tweets that can't be found in a separate list
invalid_tweet_ids = []

for tweet_id in df['tweet_id']:   
    try:
        valid_tweet_ids.append(api.get_status(tweet_id))
    except Exception as e:
        invalid_tweet_ids.append(tweet_id)

Rate limit reached. Sleeping for: 730
Rate limit reached. Sleeping for: 731


In [6]:
len(valid_tweet_ids)

2331

In [7]:
len(invalid_tweet_ids)

25

Out of the 2356 tweets only 25 are not available anymore

In [16]:
# Tweet IDs for which to gather additional data via Twitter's API
tweet_ids = df.tweet_id.values
len(tweet_ids)

2356

In [17]:
# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()

# Save each tweet's returned JSON as a new line in a .txt file
with open('Data/tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        # tweet not available
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()


1: 892420643555336193
Success
2: 892177421306343426
Success
3: 891815181378084864
Success
4: 891689557279858688
Success
5: 891327558926688256
Success
6: 891087950875897856
Success
7: 890971913173991426
Success
8: 890729181411237888
Success
9: 890609185150312448
Success
10: 890240255349198849
Success
11: 890006608113172480
Success
12: 889880896479866881
Success
13: 889665388333682689
Success
14: 889638837579907072
Success
15: 889531135344209921
Success
16: 889278841981685760
Success
17: 888917238123831296
Success
18: 888804989199671297
Success
19: 888554962724278272
Success
20: 888202515573088257
Fail
21: 888078434458587136
Success
22: 887705289381826560
Success
23: 887517139158093824
Success
24: 887473957103951883
Success
25: 887343217045368832
Success
26: 887101392804085760
Success
27: 886983233522544640
Success
28: 886736880519319552
Success
29: 886680336477933568
Success
30: 886366144734445568
Success
31: 886267009285017600
Success
32: 886258384151887873
Success
33: 8860541600590725

After retrieving the information directly from the API, we now must open the file, read the data and save it into a dataframe for an easier data handling. 
Also, we'll only keep the columns that we'll use for the analysis:
* Tweet id
* Count of retweets
* Count of favorites

In [9]:
# tweets output to a dataframe
tweet_count = []

with open('Data/tweet_json.txt','r') as file:
    for line in file:
        line_data = json.loads(line)
        tweet_count.append({'tweet_id': line_data['id'],
                        'rt_count': line_data['retweet_count'],
                        'fav_count': line_data['favorite_count']})
# assign the dataframe
df_tweet_count = pd.DataFrame(tweet_count, columns = ['tweet_id', 'rt_count', 'fav_count'])
df_tweet_count.head()

tweet_id  rt_count  fav_count
0  892420643555336193      7746      36400
1  892177421306343426      5736      31353
2  891815181378084864      3797      23623
3  891689557279858688      7909      39689
4  891327558926688256      8533      37903

### 3. Image Predictions File

A table full of image predictions (the top three only) alongside each tweet ID, image URL, and the image number that corresponded to the most confident prediction (numbered 1 to 4 since tweets can have up to four images).

This dataset has the following columns: 

tweet_id is the last part of the tweet URL after "status/" → https://twitter.com/dog_rates/status/889531135344209921

**p1** is the algorithm's #1 prediction for the image in the tweet → golden retriever

**p1_conf** is how confident the algorithm is in its #1 prediction → 95%

**p1_dog** is whether or not the #1 prediction is a breed of dog → TRUE

**p2** is the algorithm's second most likely prediction → Labrador retriever

**p2_conf** is how confident the algorithm is in its #2 prediction → 1%

**p2_dog** is whether or not the #2 prediction is a breed of dog → TRUE
etc.

This file (image_predictions.tsv) is hosted on Udacity's servers and should be downloaded programmatically using the Requests library

In [85]:
# download the tsv file
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
r = requests.get(url)

In [87]:
# write the request into a file
with open('Data/image-predictions.tsv', mode ='wb') as file:
    file.write(r.content)

In [3]:
# read the file 
df_image = pd.read_csv('Data/image-predictions.tsv', sep='\t' )

df_image.sample(5)

tweet_id                                          jpg_url  \
1974  869772420881756160  https://pbs.twimg.com/media/DBINZcxXgAQ-R6P.jpg   
406   673708611235921920  https://pbs.twimg.com/media/CVl-Z0dWcAAs7wr.jpg   
715   685547936038666240  https://pbs.twimg.com/media/CYOONfZW8AA7IOA.jpg   
1592  798673117451325440  https://pbs.twimg.com/media/CV_cnjHWUAADc-c.jpg   
5     666050758794694657  https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg   

      img_num                    p1   p1_conf  p1_dog                p2  \
1974        1              Pembroke  0.980148    True          Cardigan   
406         1      golden_retriever  0.936333    True    cocker_spaniel   
715         1              web_site  0.923987   False      oscilloscope   
1592        1                 dough  0.806757   False            bakery   
5           1  Bernese_mountain_dog  0.651137    True  English_springer   

       p2_conf  p2_dog                          p3   p3_conf  p3_dog  
1974  0.019271    True                    malamute  0.000136    True  
406   0.024211    True          Labrador_retriever  0.009435    True  
715   0.009712   False          hand-held_computer  0.008769   False  
1592  0.027907   False                 French_loaf  0.018189   False  
5     0.263788    True  Greater_Swiss_Mountain_dog  0.016199    True

## Assesing data


**Key Points**
* Only original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* Cleaning includes merging individual pieces of data according to the rules of tidy data.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This unique rating system is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



#### Twitter Archive

In [27]:
# Get a sample of the dataframe
df_archive.sample(5)

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
54    881666595344535552                    NaN                  NaN   
2017  672160042234327040                    NaN                  NaN   
1782  677687604918272002                    NaN                  NaN   
263   842535590457499648                    NaN                  NaN   
2255  667773195014021121                    NaN                  NaN   

                      timestamp  \
54    2017-07-03 00:11:11 +0000   
2017  2015-12-02 21:06:56 +0000   
1782  2015-12-18 03:11:30 +0000   
263   2017-03-17 00:38:32 +0000   
2255  2015-11-20 18:35:10 +0000   

                                                 source  \
54    <a href="http://twitter.com/download/iphone" r...   
2017  <a href="http://twitter.com/download/iphone" r...   
1782  <a href="http://twitter.com/download/iphone" r...   
263   <a href="http://twitter.com/download/iphone" r...   
2255  <a href="http://twitter.com" rel="nofollow">Tw...   

                                                   text  retweeted_status_id  \
54    This is Gary. He couldn't miss this puppertuni...                  NaN   
2017  This is Bubba. He's a Titted Peebles Aorta. Ev...                  NaN   
1782  This was Cindy's face when she heard Susan for...                  NaN   
263   This is Winnie. She lost her body saving a chi...                  NaN   
2255  This is a rare Hungarian Pinot named Jessiga. ...                  NaN   

      retweeted_status_user_id retweeted_status_timestamp  \
54                         NaN                        NaN   
2017                       NaN                        NaN   
1782                       NaN                        NaN   
263                        NaN                        NaN   
2255                       NaN                        NaN   

                                          expanded_urls  rating_numerator  \
54    https://twitter.com/dog_rates/status/881666595...                13   
2017  https://twitter.com/dog_rates/status/672160042...                 8   
1782  https://twitter.com/dog_rates/status/677687604...                11   
263   https://twitter.com/dog_rates/status/842535590...                13   
2255  https://twitter.com/dog_rates/status/667773195...                 8   

      rating_denominator    name doggo floofer  pupper puppo  
54                    10    Gary  None    None    None  None  
2017                  10   Bubba  None    None  pupper  None  
1782                  10    None  None    None    None  None  
263                   10  Winnie  None    None    None  None  
2255                  10       a  None    None    None  None

At first sight, the dataset contains dog names that are one letter long, and others are "None". 
The same happens with the stage, it's divided in four columns, when it could be just one with the stage on it. And in some cases all columns have "None" instead of empty. 

The timestamp has the +0000 at the end that is unnecesary, it should be converted to datetime type. 

From a small sample of the dataframe we can see that same dogs seem to have no name or stage. 

In [2]:
df_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

Additional to the type change of the timestamp, there two more data type changes: 
- The tweet_id should be changed to str, because it's a number too big for an integer. 
- The retweeted_status_timestamp should also changed to datetime, but it won't be relevant for the analysis
- in_reply_to_user_id should be a string, because is the user id, not a float, but it won't be relevant for the analysis

The columns source and in_reply_to_user_id also won't be relevant for the analysis. 

In [3]:
# Check empty values
df_archive.isna().sum()

tweet_id                         0
in_reply_to_status_id         2278
in_reply_to_user_id           2278
timestamp                        0
source                           0
text                             0
retweeted_status_id           2175
retweeted_status_user_id      2175
retweeted_status_timestamp    2175
expanded_urls                   59
rating_numerator                 0
rating_denominator               0
name                             0
doggo                            0
floofer                          0
pupper                           0
puppo                            0
dtype: int64

In [14]:
# count of unique values
df_archive.nunique()

tweet_id                      2356
in_reply_to_status_id           77
in_reply_to_user_id             31
timestamp                     2356
source                           4
text                          2356
retweeted_status_id            181
retweeted_status_user_id        25
retweeted_status_timestamp     181
expanded_urls                 2218
rating_numerator                40
rating_denominator              18
name                           957
doggo                            2
floofer                          2
pupper                           2
puppo                            2
dtype: int64

As said in the key points, the fact that the rating numerators are greater than the denominators does not need to be cleaned, but I'll check that there are no invalid denominators of value zero(0). 

In [39]:
sum(df_archive['rating_denominator'] == 0)

1

There's only one record with an invalid denominator, I'll check that record.

In [40]:
df_archive[df_archive['rating_denominator'] == 0]

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
313  835246439529840640           8.352460e+17           26259576.0   

                     timestamp  \
313  2017-02-24 21:54:03 +0000   

                                                source  \
313  <a href="http://twitter.com/download/iphone" r...   

                                                  text  retweeted_status_id  \
313  @jonnysun @Lin_Manuel ok jomny I know you're e...                  NaN   

     retweeted_status_user_id retweeted_status_timestamp expanded_urls  \
313                       NaN                        NaN           NaN   

     rating_numerator  rating_denominator  name doggo floofer pupper puppo  
313               960                   0  None  None    None   None  None

In [44]:
# Change pandas setting to display the full text
pd.set_option('display.max_colwidth', -1)

In [49]:
df_archive[df_archive['rating_denominator'] == 0].text

313    @jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho
Name: text, dtype: object

It's understandable that it was considered as a tweet with a rating, because it does include a rating but is not valid. This row should be discarded. 

In [8]:
# Check duplicates
df_archive.duplicated().sum()

0

In [32]:
# Count dogs with no name
df_archive[df_archive['name']=='None'].shape[0]

745

In [34]:
# Count dogs with no stage
df_archive[(df_archive['doggo']=='None') & (df_archive['floofer']=='None') & (df_archive['pupper']=='None') & (df_archive['puppo']=='None')].shape[0]

1976

In [45]:
#Count dos with more than one stage
df_count_None = df_archive.assign(count=(df_archive.loc[:, 'doggo':'puppo']!='None').sum(axis=1))

df_count_None[(df_count_None['count']>1)][['text', 'doggo','floofer','pupper','puppo']]

text  \
191   Here's a puppo participating in the #ScienceMarch. Cleverly disguising her own doggo agenda. 13/10 would keep the planet habitable for https://t.co/cMhq16isel         
200   At first I thought this was a shy doggo, but it's actually a Rare Canadian Floofer Owl. Amateurs would confuse the two. 11/10 only send dogs https://t.co/TXdT3tmuYk   
460   This is Dido. She's playing the lead role in "Pupper Stops to Catch Snow Before Resuming Shadow Box with Dried Apple." 13/10 (IG: didodoggo) https://t.co/m7isZrOBX7   
531   Here we have Burke (pupper) and Dexter (doggo). Pupper wants to be exactly like doggo. Both 12/10 would pet at same time https://t.co/ANBpEYHaho                       
565   Like doggo, like pupper version 2. Both 11/10 https://t.co/9IxWAXFqze                                                                                                  
575   This is Bones. He's being haunted by another doggo of roughly the same size. 12/10 deep breaths pupper everything's fine https://t.co/55Dqe0SJNj                       
705   This is Pinot. He's a sophisticated doggo. You can tell by the hat. Also pointier than your average pupper. Still 10/10 would pet cautiously https://t.co/f2wmLZTPHd   
733   Pupper butt 1, Doggo 0. Both 12/10 https://t.co/WQvcPEpH2u                                                                                                             
778   RT @dog_rates: Like father (doggo), like son (pupper). Both 12/10 https://t.co/pG2inLaOda                                                                              
822   RT @dog_rates: This is just downright precious af. 12/10 for both pupper and doggo https://t.co/o5J479bZUC                                                             
889   Meet Maggie &amp; Lila. Maggie is the doggo, Lila is the pupper. They are sisters. Both 12/10 would pet at the same time https://t.co/MYwR4DQKll                       
956   Please stop sending it pictures that don't even have a doggo or pupper in them. Churlish af. 5/10 neat couch tho https://t.co/u2c9c7qSg8                               
1063  This is just downright precious af. 12/10 for both pupper and doggo https://t.co/o5J479bZUC                                                                            
1113  Like father (doggo), like son (pupper). Both 12/10 https://t.co/pG2inLaOda                                                                                             

      doggo  floofer  pupper  puppo  
191   doggo  None     None    puppo  
200   doggo  floofer  None    None   
460   doggo  None     pupper  None   
531   doggo  None     pupper  None   
565   doggo  None     pupper  None   
575   doggo  None     pupper  None   
705   doggo  None     pupper  None   
733   doggo  None     pupper  None   
778   doggo  None     pupper  None   
822   doggo  None     pupper  None   
889   doggo  None     pupper  None   
956   doggo  None     pupper  None   
1063  doggo  None     pupper  None   
1113  doggo  None     pupper  None

It looks like some tweets refer to two dogs, and others have more than one stage extracted from the text because of the word game. 

#### Tweet image predictions 

In [50]:
df_image.sample(5)

tweet_id                                          jpg_url  \
137   668528771708952576  https://pbs.twimg.com/media/CUcXXpxWUAAUJ__.jpg   
1453  776819012571455488  https://pbs.twimg.com/media/CW88XN4WsAAlo8r.jpg   
385   673345638550134785  https://pbs.twimg.com/media/CVg0SVRWEAAsBrS.jpg   
1278  750101899009982464  https://pbs.twimg.com/media/Cmjlsh1XgAEvhq_.jpg   
704   684926975086034944  https://pbs.twimg.com/media/CYFZXdiU0AAc_kw.jpg   

      img_num                  p1   p1_conf  p1_dog                p2  \
137   1        Labrador_retriever  0.195835  True    kuvasz             
1453  3        Chihuahua           0.346545  True    dalmatian          
385   1        hamster             0.761025  False   weasel             
1278  2        golden_retriever    0.316704  True    llama              
704   1        Labrador_retriever  0.769412  True    golden_retriever   

       p2_conf  p2_dog                  p3   p3_conf  p3_dog  
137   0.121607  True    English_setter      0.081464  True    
1453  0.166246  True    toy_terrier         0.117502  True    
385   0.069362  False   Pomeranian          0.064627  True    
1278  0.174269  False   Labrador_retriever  0.147364  True    
704   0.144893  True    lion                0.021440  False

From a sample of the dataset we can see that some of the recors may not correspond to dogs. And we don't need three predictions assigned to each image, so these values must only comprise one column with the non-dog value with the higher confidence score. 

Also, the dogs names are not consistent, some of them start with capital letters and other dont. 

In [11]:
df_image.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


From the info of the dataset, we can see that it has almost 300 records less than the 

In [12]:
df_image.isna().sum()

tweet_id    0
jpg_url     0
img_num     0
p1          0
p1_conf     0
p1_dog      0
p2          0
p2_conf     0
p2_dog      0
p3          0
p3_conf     0
p3_dog      0
dtype: int64

In [13]:
df_image.duplicated().sum()

0

Other than the first observation, the dataset is quite clean, no duplicates or missing values. 

#### Twitter count & favs data

In [51]:
# Get a sample of the dataset
df_tweet_count.sample(5)

tweet_id  rt_count  fav_count
144   863062471531167744  2300      8139     
585   798628517273620480  2029      0        
2182  668627278264475648  109       307      
1370  700002074055016451  1344      3331     
2265  667171260800061440  80        212

This dataset is very simple and pretty straight-forward. The only issue that I expect to find is an amount of records lower than the one from the previous datasets, because some tweets were not available anymore. 

In [15]:
df_tweet_count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 3 columns):
tweet_id     2331 non-null int64
rt_count     2331 non-null int64
fav_count    2331 non-null int64
dtypes: int64(3)
memory usage: 54.7 KB


The dataset prensents the same issue with tweet_id datatype and the amount of rows is less that the twitter archive, but still more that the image predictions dataset. 

In [16]:
# Check for missing values
df_tweet_count.isna().sum()

tweet_id     0
rt_count     0
fav_count    0
dtype: int64

In [52]:
# Check for duplicated rows
df_tweet_count.duplicated().sum()

0

In [2]:
#Check that all values are positive
(df_tweet_count.rt_count.min(),df_tweet_count.fav_count.min())

(1, 0)

In [20]:
# Check for duplicates
df_tweet_count.tweet_id.duplicated().sum()

0

### Quality Issues

As a summary of what I've encountered in the assesment of the data, the quality issues are the following:

**Twitter archive**
- Some fields have the wrong datatype and should be corrected:
    * tweet_id must be string
    * timestamp must be datetime
- Some dog names have "None" as a name instead of empty value (Nan)
- One record has an invalid denominator, equal to zero and should be discarded
- Empty dog type have "None" as a name instead of empty value (Nan)
- The dog names starts with upper or lowercase indistinctly

**Tweet image predictions**
- The field tweet_id must be string
- Some predictions do not correspond to dog
- The dog type starts with upper or lowercase indistinctly

**Twitter count & favs data**
- The field tweet_id must be string


### Tidiness Issues

As a summary of what I've encountered in the assesment of the data, the tidiness issues are the following:

**Twitter archive**
- The dog stages is spread in 4 columns, must be merged in one column with the different values
- Some records have more than on dog stage assigned and should be drop
- The columns retweeted_status_timestamp,retweeted_status_user_id, in_reply_to_user_id, source and in_reply_to_user_id won't be relevant for the analysis. 

**Tweet image predictions**

- There are 3 predictions for each record, these values must only comprise one column with the non-dog value with the higher confidence score.

**Twitter count & favs data**


**General Issues**
- All datasets have different amount of recors, this must be unified
- Each dataset has an observation on the same record, they must be combined in one clean dataset using the key tweet_id. 

## Cleaning data

Correct datatypes

In [60]:
#The field tweet_id must be string in all data frames
df_archive.tweet_id = df_archive.tweet_id.astype(str)
df_image.tweet_id = df_image.tweet_id.astype(str)
df_tweet_count.tweet_id = df_tweet_count.tweet_id.astype(str)

#timestamp and retweeted_status_timestamp should be datetime
df_archive['timestamp']= pd.to_datetime(df_archive['timestamp']) 

Drop records with invalid denominator

In [61]:
df_archive.drop(df_archive.index[['rating_denominator'] == 0], inplace=True)

Drop records with more than one dog stage assigned

In [62]:
# Count the records that are not None
df_archive = df_archive.assign(count=(df_archive.loc[:, 'doggo':'puppo']!='None').sum(axis=1))

# drop the records with a count greater than 1
df_archive.drop(df_archive.index[(df_archive['count']>1)], inplace=True)


# drop the auxiliar count column
df_archive.drop(['count'], axis=1, inplace=True);

Replace the string value "None" with the empty value (Nan) in columns dog name and dog type

In [63]:
df_archive.replace(to_replace='([Nn]one)', value=np.nan, regex=True, inplace=True)

Combine dog types in one column

In [66]:
df_archive['dog_stage'] = np.where(df_archive['doggo'].isna() == False, df_archive['doggo'], df_archive['floofer'] )
df_archive['dog_stage'] = np.where(df_archive['dog_stage'].isna() == False, df_archive['dog_stage'], df_archive['pupper'] )
df_archive['dog_stage'] = np.where(df_archive['dog_stage'].isna() == False, df_archive['dog_stage'], df_archive['puppo'] )

# drop the dog stages columns, that have been replaced now
df_archive.drop(['doggo','floofer', 'pupper', 'puppo'], axis=1, inplace=True);

Set all names and breeds to all lower case letter

In [67]:
df_archive['name'] = df_archive['name'].str.lower()
df_archive['dog_stage'] = df_archive['dog_stage'].str.lower()

Drop columns that won't be relevant for the analysis

In [109]:
df_archive.drop(['retweeted_status_timestamp','retweeted_status_user_id','in_reply_to_user_id', 'in_reply_to_user_id','source'], axis=1, inplace=True); 

In [115]:
# Check the changes
df_archive.sample(3)

tweet_id  in_reply_to_status_id           timestamp  \
1545  689289219123089408                    NaN 2016-01-19 03:32:10   
1197  717009362452090881                    NaN 2016-04-04 15:22:08   
749   779124354206535695                    NaN 2016-09-23 01:04:56   

                                                   text  retweeted_status_id  \
1545  Someone sent me this without any context and e...                  NaN   
1197  This is Smokey. He's having some sort of exist...                  NaN   
749   RT @dog_rates: Meet Hurley. He's the curly one...         6.794628e+17   

                                          expanded_urls  rating_numerator  \
1545  https://twitter.com/dog_rates/status/689289219...                13   
1197  https://twitter.com/dog_rates/status/717009362...                10   
749   https://twitter.com/dog_rates/status/679462823...                11   

      rating_denominator    name dog_breed  
1545                  10     NaN       NaN  
1197                  10  smokey    pupper  
749                   10  hurley       NaN

Get the prediction with the highest confidence that is a dog. 

In [116]:
# Replace non dog results with confidence 0 so they don't get picked as the highest result
df_image['p1_conf'] = np.where(df_image['p1_dog']==False, 0, df_image['p1_conf'])
df_image['p2_conf'] = np.where(df_image['p2_dog']==False, 0, df_image['p2_conf'])
df_image['p3_conf'] = np.where(df_image['p3_dog']==False, 0, df_image['p3_conf'])

# Add a column with the highest confidence
df_image['max_conf'] = df_image[['p1_conf','p2_conf','p3_conf']].max(axis=1)
#.idxmax(axis=1)

#get the dog result with the max confidence for each row
df_image['dog_breed'] = np.where(df_image['p1_conf'] ==  df_image['max_conf'] , df_image['p1'], np.nan )
df_image['dog_breed'] = np.where(df_image['p2_conf'] ==  df_image['max_conf'] , df_image['p2'], df_image['dog_breed'] )
df_image['dog_breed'] = np.where(df_image['p3_conf'] ==  df_image['max_conf'] , df_image['p3'], df_image['dog_breed']  )

# drop the dog stages columns, that have been replaced now
df_image.drop(['max_conf','p1','p1_conf', 'p1_dog', 'p2','p2_conf', 'p2_dog','p3','p3_conf', 'p3_dog',], axis=1, inplace=True);

In [ ]:
# set all values to lower case
df_image['dog_breed'] = df_image['dog_breed'].str.lower()

In [117]:
# Drop unused columns
df_image.drop(['img_num'], axis=1, inplace=True);

In [118]:
# check the change
df_image.head()

tweet_id                                          jpg_url  \
0  666020888022790149  https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1  666029285002620928  https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   
2  666033412701032449  https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg   
3  666044226329800704  https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg   
4  666049248165822465  https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg   

              dog_breed_p  
0  Welsh_springer_spaniel  
1                 redbone  
2         German_shepherd  
3     Rhodesian_ridgeback  
4      miniature_pinscher

Merge all three datasets into one using the tweet_id existing in each dataset

In [123]:
df = pd.merge(df_archive,df_image, on ='tweet_id', how="inner")
df = pd.merge(df, df_tweet_count, on ='tweet_id', how="inner")

In [131]:
# check the new structure
df.sample(3)

tweet_id  in_reply_to_status_id           timestamp  \
689   761292947749015552                    NaN 2016-08-04 20:09:17   
1072  706681918348251136                    NaN 2016-03-07 03:24:33   
1956  667530908589760512                    NaN 2015-11-20 02:32:25   

                                                   text  retweeted_status_id  \
689   Meet Bonaparte. He's pupset because it's cloud...                  NaN   
1072  Say hello to Katie. She's a Mitsubishi Hufflep...                  NaN   
1956  Meet Naphaniel. He doesn't necessarily enjoy h...                  NaN   

                                          expanded_urls  rating_numerator  \
689   https://twitter.com/dog_rates/status/761292947...                11   
1072  https://twitter.com/dog_rates/status/706681918...                12   
1956  https://twitter.com/dog_rates/status/667530908...                10   

      rating_denominator       name dog_breed  \
689                   10  bonaparte       NaN   
1072                  10      katie       NaN   
1956                  10  naphaniel       NaN   

                                              jpg_url       dog_breed_p  \
689   https://pbs.twimg.com/media/CpCn5aXXgAAOPTm.jpg   standard_poodle   
1072  https://pbs.twimg.com/media/Cc6jcYRXIAAFuox.jpg        toy_poodle   
1956  https://pbs.twimg.com/media/CUOL0uGUkAAx7yh.jpg  golden_retriever   

      rt_count  fav_count  
689       1097       4537  
1072       973       3306  
1956       227        448

In [195]:
#df[df.dog_breed.notna()].dog_breed_p.notna()
df[((df['dog_breed'].notna()) == (df['dog_breed_p'].notna())) == True]

tweet_id  in_reply_to_status_id           timestamp  \
8     890240255349198849                    NaN 2017-07-26 15:59:51   
11    889665388333682689                    NaN 2017-07-25 01:55:32   
13    889531135344209921                    NaN 2017-07-24 17:02:04   
27    886366144734445568                    NaN 2017-07-15 23:25:31   
37    884162670584377345                    NaN 2017-07-09 21:29:42   
40    883360690899218434                    NaN 2017-07-07 16:22:55   
43    882762694511734784                    NaN 2017-07-06 00:46:41   
49    881536004380872706                    NaN 2017-07-02 15:32:16   
62    878776093423087618                    NaN 2017-06-25 00:45:22   
66    877611172832227328                    NaN 2017-06-21 19:36:23   
70    876838120628539392                    NaN 2017-06-19 16:24:33   
77    874296783580663808                    NaN 2017-06-12 16:06:11   
79    874012996292530176                    NaN 2017-06-11 21:18:31   
81    873213775632977920                    NaN 2017-06-09 16:22:42   
82    872967104147763200                    NaN 2017-06-09 00:02:31   
88    871762521631449091                    NaN 2017-06-05 16:15:56   
89    871515927908634625                    NaN 2017-06-04 23:56:03   
98    869596645499047938                    NaN 2017-05-30 16:49:31   
105   867421006826221569                    NaN 2017-05-24 16:44:18   
109   866450705531457537                    NaN 2017-05-22 00:28:40   
136   859607811541651456                    NaN 2017-05-03 03:17:27   
139   858843525470990336                    NaN 2017-05-01 00:40:27   
154   854120357044912130                    NaN 2017-04-17 23:52:16   
171   850019790995546112                    NaN 2017-04-06 16:18:05   
187   846514051647705089                    NaN 2017-03-28 00:07:32   
192   845397057150107648                    NaN 2017-03-24 22:08:59   
193   845306882940190720                    NaN 2017-03-24 16:10:40   
217   839290600511926273                    NaN 2017-03-08 01:44:07   
224   837820167694528512                    NaN 2017-03-04 00:21:08   
227   837110210464448512                    NaN 2017-03-02 01:20:01   
...                  ...                    ...                 ...   
1574  675113801096802304                    NaN 2015-12-11 00:44:07   
1579  675006312288268288                    NaN 2015-12-10 17:37:00   
1588  674774481756377088                    NaN 2015-12-10 02:15:47   
1595  674737130913071104                    NaN 2015-12-09 23:47:22   
1601  674638615994089473                    NaN 2015-12-09 17:15:54   
1604  674447403907457024                    NaN 2015-12-09 04:36:06   
1611  674318007229923329                    NaN 2015-12-08 20:01:55   
1616  674262580978937856                    NaN 2015-12-08 16:21:41   
1625  674038233588723717                    NaN 2015-12-08 01:30:12   
1631  673956914389192708                    NaN 2015-12-07 20:07:04   
1632  673919437611909120                    NaN 2015-12-07 17:38:09   
1639  673707060090052608                    NaN 2015-12-07 03:34:14   
1642  673697980713705472                    NaN 2015-12-07 02:58:09   
1648  673656262056419329                    NaN 2015-12-07 00:12:23   
1650  673612854080196609                    NaN 2015-12-06 21:19:54   
1654  673363615379013632                    NaN 2015-12-06 04:49:31   
1661  673342308415348736                    NaN 2015-12-06 03:24:51   
1664  673295268553605120                    NaN 2015-12-06 00:17:55   
1668  673148804208660480                    NaN 2015-12-05 14:35:56   
1671  672988786805112832                    NaN 2015-12-05 04:00:04   
1674  672975131468300288                    NaN 2015-12-05 03:05:49   
1675  672970152493887488                    NaN 2015-12-05 02:46:02   
1679  672898206762672129                    NaN 2015-12-04 22:00:08   
1685  672622327801233409                    NaN 2015-12-04 03:43:54   
1686  672614745925664768                

## Storing the clean data

In [135]:
df.to_csv('Data/twitter_archive_master.csv')

## Analyze and visualize the data
At least three (3) insights and one (1) visualization must be produced.

In [ ]:
#df = pd.read_csv('Data/twitter_archive_master.csv')

In [186]:
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt
% matplotlib inline


# form one text variable with all dog_breeds
text = " ".join(str(text) for text in df.dog_breed_p)

# Generate a word cloud image
wordcloud = WordCloud(max_font_size=100, max_words=50, stopwords=stopwords, background_color="white").generate(text)

# store to file
wc.to_file("Data/dog_word_cloud.png")

# show
plt.figure(figsize=[20,10])
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()


## Reperences
- [Tweet object overview](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object)
- [Tidy Data in Python article by Jean-Nicholas Hould](http://www.jeannicholashould.com/tidy-data-in-python.html)